# RAG Workshop #1 質問をTransform


## データを準備

### 使用データ
wikivoyageにある日本の観光地をURLでスクレイピングする

### ChromaDB というVector DBを使用してデータをEmbedding

In [ ]:
from langchain_chroma import Chroma
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
import chromadb

In [ ]:
client = chromadb.PersistentClient(path="./my_chroma_db")

granular_collection = Chroma(
    client=client,
    collection_name="granular",
    embedding_function=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2"),
)

granular_collection.reset_collection()

### HTMLSectionSplitterを使ってHTMLをスクレイプし、chromadbの中にINGESTする

In [ ]:
from langchain_text_splitters import HTMLSectionSplitter
from langchain_community.document_loaders import AsyncHtmlLoader

In [ ]:
destinations = [
    "Tokyo",
    "Hiroshima",
    "Kanazawa",
    "Kyoto",
    "Nagasaki",
    "Nara",
    "Osaka",
    "Sapporo",
    "Sendai",
]

wikivoyage_root_url = "https://en.wikivoyage.org/wiki"


In [ ]:
destination_urls = [f"{wikivoyage_root_url}/{d}" for d in destinations]

In [ ]:
headers_to_split_on = [("h1", "Header 1"), ("h2", "Header 2")]
html_section_splitter = HTMLSectionSplitter(headers_to_split_on=headers_to_split_on)

In [ ]:
def split_docs_into_granular_chunks(docs):
    all_chunks = []
    for doc in docs:
        html_string = doc.page_content
        temp_chunks = html_section_splitter.split_text(html_string)
        h2_temp_chunks = [
            chunk for chunk in temp_chunks if "Header 2" in chunk.metadata
        ]
        all_chunks.extend(h2_temp_chunks)

    return all_chunks


In [ ]:
for destination_url in destination_urls:
    html_loader = AsyncHtmlLoader(destination_url)
    docs = html_loader.load()

    for doc in docs:
        print(doc.metadata)
        granular_chunks = split_docs_into_granular_chunks(docs)
        granular_collection.add_documents(documents=granular_chunks)


## Rewrite-retrieve-readを使おう

### 従来のRAGの問題点
- ユーザーの質問が不明瞭な場合、適切なドキュメントを検索できないことがある。
- それを解決するために生まれたのが Rewrite-Retrieve-Readという手法

### ユーザーからの質問でドキュメントを検索してみる

-  Scoreは高いほど検索条件により近い

In [ ]:
user_question = "北海道で楽しめることは？"
improved_results = granular_collection.similarity_search_with_score(
    query=user_question, k=4
)
for i, (doc, score) in enumerate(improved_results):
    print("-" * 100)
    print(f"🐯 Result {i + 1}:")
    print(f"💫 Score: {score:.2f}")
    print(doc)

結果： 質問が悪いと引っ張ってこられるChunkもひどい

### ユーザーの質問を書き換えたらどうなるか…

### 質問を書き換えてくれる Rewriter chain を setup

In [ ]:
from langchain_ollama.chat_models import ChatOllama
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

In [ ]:
llm = ChatOllama(model="gemma3:4b")

‼️ ユーザーからのインプットは日本語、最終アウトプットは日本、内部は英語が望ましい。

In [ ]:
rewriter_prompt_template = """
Generate search query for the Chroma DB vector store from a user question, allowing for a more accurate response through semantic search.
Just return the revised Chroma DB query, with quotes around it. 

User question: {user_question}
Revised Chroma DB query:
"""

rewriter_prompt = ChatPromptTemplate.from_template(rewriter_prompt_template)

In [ ]:
rewriter_chain = rewriter_prompt | llm | StrOutputParser()


### 書き直された質問で情報をchromadbから回収する

In [ ]:
search_query = rewriter_chain.invoke({"user_question": user_question})
print(search_query)

In [ ]:
improved_results = granular_collection.similarity_search_with_score(
    query=user_question, k=4
)
for i, (doc, score) in enumerate(improved_results):
    print("-" * 100)
    print(f"🐯 Result {i + 1}:")
    print(f"💫 Score: {score:.2f}")
    print(doc)

###  RAG Chain を作成

In [ ]:
from langchain_core.runnables import RunnablePassthrough

In [ ]:
retriever = granular_collection.as_retriever()

rag_prompt_template = """
Given a question and some context, answer the question.
If you do not know the answer, just say I do not know.
Answer in bullet points in Japanese.


Context: {context}
Question: {question}
"""

rag_prompt = ChatPromptTemplate.from_template(rag_prompt_template)

rewrite_retrieve_read_rag_chain = (
    {
        "context": {"user_question": RunnablePassthrough()}
        | rewriter_chain
        | retriever,
        "question": RunnablePassthrough(),
    }
    | rag_prompt
    | llm
    | StrOutputParser()
)


In [ ]:
answer = rewrite_retrieve_read_rag_chain.invoke(user_question)
print(answer)

### 結果
- まだまだ改良の余地がある。
- 質問がひどい場合の対応をしているだけ。

## Multiple query generation を使ってより正確にVector DB からDocumentをRetrieveしよう


### 従来のRAGの問題点
- ユーザーの質問にあるキーワードにだけ近い近似値スコアでDocumentをRetrieveしてしまう。
- それを解消するための手法が Multiple query generation

In [ ]:
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain_core.prompts import ChatPromptTemplate

from typing import List
from langchain_core.output_parsers import BaseOutputParser
from pydantic import BaseModel, Field

### MultiQueryRetriver の実装

### Promptの設定

In [ ]:
multi_query_gen_prompt_template = """
You are an AI language model assistant. Your task is to generate five 
different versions of the given user question to retrieve relevant documents from a vector 
database. By generating multiple perspectives on the user question, your goal is to help
the user overcome some of the limitations of the distance-based similarity search. 
Provide these alternative questions separated by newlines.
Original question: {question}
"""

multi_query_gen_prompt = ChatPromptTemplate.from_template(
    multi_query_gen_prompt_template
)

### multi-query parser の設定

In [ ]:
class LineListOutputParser(BaseOutputParser[List[str]]):
    """Parse out a question from each output line."""

    def parse(self, text: str) -> List[str]:
        lines = text.strip().split("\n")
        return list(filter(None, lines))


questions_parser = LineListOutputParser()

### 質問(Query)を複数作る Chain の作成

In [ ]:
multi_query_gen_chain = multi_query_gen_prompt | llm | questions_parser

### どのようなQueryが生成されるかを確認

In [ ]:
multiple_queries = multi_query_gen_chain.invoke(user_question)
multiple_queries

### MultiQueryRetrieverを設定

In [ ]:
basic_retriever = granular_collection.as_retriever()

multi_query_retriever = MultiQueryRetriever(
    retriever=basic_retriever,
    llm_chain=multi_query_gen_chain,
    parser_key="lines",
)

### multi_query retrieverの出力を確認

In [ ]:
retrieved_docs = multi_query_retriever.invoke(user_question)
retrieved_docs

# Step-back question の実装

### 従来のRAGの問題点
- 詳細な質問をしたときに、特定のものを検索してしまうことによって、RetrieveできるDocumentが減ってしまう。
- そのために生まれた手法が Step-back Question

### Step-back question を生成するための準備

In [ ]:
step_back_prompt_template = """
Generate a less specific question (aka Step-back question) for the following detailed question, so that a wider context can be retrieved.
Detailed question: {detailed_question}
Step-back question:
"""

step_back_prompt = ChatPromptTemplate.from_template(step_back_prompt_template)

In [ ]:
step_back_question_gen_chain = step_back_prompt | llm | StrOutputParser()

### step-back-question generation chain によって得られる質問を確認

In [ ]:
step_back_question = step_back_question_gen_chain.invoke(user_question)

In [ ]:
step_back_question

### Step-back question をRAG chain に導入する

In [ ]:
retriever = granular_collection.as_retriever()

rag_prompt_template = """
Given a question and some context, answer the question.
If you do not know the answer, just say I do not know.
If the information you get is in English, translate it to Japanese.
You must answer in bullet points in Japanese.
Give me as much detail as possible.

Context: {context}
Question: {question}
"""

rag_prompt = ChatPromptTemplate.from_template(rag_prompt_template)

step_back_question_rag_chain = (
    {
        "context": {"detailed_question": RunnablePassthrough()}
        | step_back_question_gen_chain
        | retriever,
        "question": RunnablePassthrough(),
    }
    | rag_prompt
    | llm
    | StrOutputParser()
)
user_question

In [ ]:
answer = step_back_question_rag_chain.invoke(user_question)
print(answer)

##  Rewrite-Retrieve-Readの利点
- ざっくりの質問からよりVector DBが検索しやすい、正確な近しいDocumentをRetrieveしてくれるような質問に書き換えてくれる

## Multi Query Generationの利点
- ユーザーに質問から複数の質問を色々な角度から生成してくることにより、近似値計算の弊害を弱めてより近しいDocumentをRetrieveするようにしてくれる
## Step back Questionの利点
- 詳しすぎる質問をしたときに、多くのDocumentをRetriveしてくれるようにスコープを広げた質問に変換してくれる

## 問題点は？
- 同じ質問をしても、答えの質がばらばら（non-deterministic）
  - 解決策： LoRAなどのチューニングが必要 
- 質問によって手法を変える必要がある
  - 解決策： LangGraphなどで条件分岐を行う必要がある 
- そもそも、間違った土地の情報が入ってしまう
  - 解決策： Chunkのなかにメタキーワードを埋め込み、フィルターする必要がある 